# LSTM IMDB Movie Review Tutorial
Josiah Olson

In [1]:
from __future__ import print_function
import time
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, merge, BatchNormalization, GRU
from keras.datasets import imdb

import os
from keras.preprocessing.text import Tokenizer

Using Theano backend.


In [2]:
max_features = 10000
max_len = 30  # cut texts after this number of words (among top max_features most common words)

In [3]:
# get dataset and unzip: http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

X_train = []
y_train = []

path = 'C:/1_Research/Create_data/aclImdb/train_sukien/pos/'
X_train.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_train.extend([1 for _ in range(1998)])

path = 'C:/1_Research/Create_data/aclImdb/train_sukien/neg/'
X_train.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_train.extend([0 for _ in range(782)])

print('x:')
print(X_train[:1])
print(X_train[-1:])
print(len(X_train))
print('y:')
print(y_train[:1])
print(y_train[-1:])
print(len(y_train))

x:
['\xef\xbb\xbfABI: Ng\xc3\xa0y GDKHQ T\xe1\xbb\x95 ch\xe1\xbb\xa9c \xc4\x90\xe1\xba\xa1i h\xe1\xbb\x99i \xc4\x91\xe1\xbb\x93ng c\xe1\xbb\x95 \xc4\x91\xc3\xb4ng th\xc6\xb0\xe1\xbb\x9dng ni\xc3\xaan n\xc4\x83m 2014 v\xc3\xa0 T\xe1\xba\xa1m \xe1\xbb\xa9ng c\xe1\xbb\x95 t\xe1\xbb\xa9c n\xc4\x83m 2013 b\xe1\xba\xb1ng ti\xe1\xbb\x81n (10%)\n']
['\xef\xbb\xbfPGT: Ng\xc3\xa0y GDKHQ tham d\xe1\xbb\xb1 \xc4\x90\xe1\xba\xa1i h\xe1\xbb\x99i \xc4\x91\xe1\xbb\x93ng c\xe1\xbb\x95 \xc4\x91\xc3\xb4ng th\xc6\xb0\xe1\xbb\x9dng ni\xc3\xaan 2016\n']
2780
y:
[1]
[0]
2780


In [4]:
# read in the test data

X_test = []
y_test = []

path = 'C:/1_Research/Create_data/aclImdb/test_sukien/pos/'
X_test.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_test.extend([1 for _ in range(999)])

path = 'C:/1_Research/Create_data/aclImdb/test_sukien/neg/'
X_test.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_test.extend([0 for _ in range(391)])

print('x:')
print(X_test[:1])
print(X_test[-1:])
print(len(X_test))
print('y:')
print(y_test[:1])
print(y_test[-1:])
print(len(y_test))

x:
['\xef\xbb\xbfSPD: Ng\xc3\xa0y GDKHQ Chi tr\xe1\xba\xa3 c\xe1\xbb\x95 t\xe1\xbb\xa9c b\xe1\xba\xb1ng ti\xe1\xbb\x81n (8%)\n']
['\xef\xbb\xbfYBC: Ng\xc3\xa0y GDKHQ tham d\xe1\xbb\xb1 \xc4\x90\xe1\xba\xa1i h\xe1\xbb\x99i \xc4\x91\xe1\xbb\x93ng c\xe1\xbb\x95 \xc4\x91\xc3\xb4ng th\xc6\xb0\xe1\xbb\x9dng ni\xc3\xaan 2016\n']
1390
y:
[1]
[0]
1390


In [5]:
#tokenize works to list of integers where each integer is a key to a word
imdbTokenizer = Tokenizer(nb_words=max_features)

imdbTokenizer.fit_on_texts(X_train)

In [6]:
#print top 20 words 
#note zero is reserved for non frequent words
for word, value in imdbTokenizer.word_index.items():
    if value < 20:
        print(value, word)

5 đông
3 gdkhq
8 Đại
4 năm
19 dự
12 tức
15 2015
13 2014
1 cổ
6 thường
7 hội
2 ngày
10 đồng
16 chức
18 trả
17 tổ
14 tiền
11 bằng
9 niên


In [7]:
#create int to word dictionary
intToWord = {}
for word, value in imdbTokenizer.word_index.items():
    intToWord[value] = word

#add a symbol for null placeholder
intToWord[0] = "!!!NA!!!"
    
print(intToWord[1])
print(intToWord[2])
print(intToWord[32])

cổ
ngày
10


In [8]:
#convert word strings to integer sequence lists
print(X_train[0])
print(imdbTokenizer.texts_to_sequences(X_train[:1]))
for value in imdbTokenizer.texts_to_sequences(X_train[:1])[0]:
    print(intToWord[value])
    
X_train = imdbTokenizer.texts_to_sequences(X_train)
X_test = imdbTokenizer.texts_to_sequences(X_test)

﻿ABI: Ngày GDKHQ Tổ chức Đại hội đồng cổ đông thường niên năm 2014 và Tạm ứng cổ tức năm 2013 bằng tiền (10%)

[[596, 2, 3, 17, 16, 8, 7, 10, 1, 5, 6, 9, 4, 13, 27, 24, 23, 1, 12, 4, 26, 11, 14, 32]]
﻿abi
ngày
gdkhq
tổ
chức
Đại
hội
đồng
cổ
đông
thường
niên
năm
2014
và
tạm
ứng
cổ
tức
năm
2013
bằng
tiền
10


In [9]:
# Censor the data by having a max review length (in number of words)

#use this function to load data from keras pickle instead of munging as shown above
#(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features,
#                                                      test_split=0.2)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

2780 train sequences
1390 test sequences
Pad sequences (samples x time)
X_train shape: (2780L, 30L)
X_test shape: (1390L, 30L)


In [10]:
#example of a sentence sequence, note that lower integers are words that occur more commonly
print("x:", X_train[0]) #per observation vector of 20000 words
print("y:", y_train[0]) #positive or negative review encoding

x: [  0   0   0   0   0   0 596   2   3  17  16   8   7  10   1   5   6   9
   4  13  27  24  23   1  12   4  26  11  14  32]
y: 1


In [11]:
# double check that word sequences behave/final dimensions are as expected
print("y distribution:", np.unique(y_train, return_counts=True))
print("max x word:", np.max(X_train), "; min x word", np.min(X_train))
print("y distribution test:", np.unique(y_test, return_counts=True))
print("max x word test:", np.max(X_test), "; min x word", np.min(X_test))

y distribution: (array([0, 1]), array([ 782, 1998], dtype=int64))
max x word: 1101 ; min x word 0
y distribution test: (array([0, 1]), array([391, 999], dtype=int64))
max x word test: 1070 ; min x word 0


In [12]:
print("most and least popular words: ")
print(np.unique(X_train, return_counts=True))
# as expected zero is the highly used word for words not in index

most and least popular words: 
(array([   0,    1,    2, ..., 1099, 1100, 1101]), array([43926,  3138,  2732, ...,     1,     1,     1], dtype=int64))


In [13]:
#set model hyper parameters
epochs = 16
embedding_neurons = 128
lstm_neurons = 64
batch_size = 32

Next let's import pre-trained word vectors from google and use them to initialize our embedding to see if this improves the neural net's performance.

In [14]:
from gensim.models import Word2Vec

#get pre trained word2vec from google:
#https://doc-0k-4g-docs.googleusercontent.com/docs/securesc/gnqvgap6hjncpd3b10i2tv865io48jas/hmjtdgee48c14e1parufukrpkb8urra5/1463018400000/06848720943842814915/09676831593570546402/0B7XkCwpI5KDYNlNUTTlSS21pQmM?e=download&nonce=4l49745nmtine&user=09676831593570546402&hash=i2qe9mshan4mesl112ct9bu1tj9kr1hq

googlew2v = Word2Vec.load_word2vec_format('C:/1_Research/Create_data/aclImdb/word2vector/GoogleNews-vectors-negative300.bin', binary=True)

C:\Anaconda2\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Anaconda2\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [15]:
# get word vectors for words in my index
googleVecs = []
for value in range(max_features):
    try:
        googleVecs.append(googlew2v[intToWord[value]])
    except:
        googleVecs.append(np.random.uniform(size=300))

googleVecs = np.array(googleVecs)

print(googleVecs)
print(googleVecs.shape)

[[ 0.26202468  0.15868397  0.27812652 ...,  0.10937801  0.75767728
   0.4428527 ]
 [ 0.96970407  0.12253514  0.35782362 ...,  0.97491444  0.6667405
   0.40817497]
 [ 0.34841225  0.36222138  0.90096367 ...,  0.47904751  0.85001801
   0.84470638]
 ..., 
 [ 0.37284314  0.53258785  0.26119324 ...,  0.88342727  0.14764413
   0.41580886]
 [ 0.83421612  0.39597201  0.98788126 ...,  0.01348185  0.10617085
   0.5794498 ]
 [ 0.18089266  0.15663464  0.0585371  ...,  0.95018796  0.15530301
   0.5283831 ]]
(10000L, 300L)


In [16]:
# Bi-directional google

# this example tests if using pretrained embeddings will improve performance 
# relative to starting with random embeddings

# this is the placeholder tensor for the input sequences
sequence = Input(shape=(max_len,), dtype='int32')
# this embedding layer will transform the sequences of integers
# into vectors of size embedding
# embedding layer converts dense int input to one-hot in real time to save memory
embedded = Embedding(max_features, 300, input_length=max_len, weights=[googleVecs])(sequence)
# normalize embeddings by input/word in sentence
bnorm = BatchNormalization()(embedded)

# apply forwards LSTM layer size lstm_neurons
forwards = GRU(lstm_neurons, dropout_W=0.8, dropout_U=0.8)(bnorm)
# apply backwards LSTM
backwards = GRU(lstm_neurons, dropout_W=0.8, dropout_U=0.8, go_backwards=True)(bnorm)

# concatenate the outputs of the 2 LSTMs
merged = merge([forwards, backwards], mode='concat', concat_axis=-1)
after_dp = Dropout(0.8)(merged)
output = Dense(1, activation='sigmoid')(after_dp)

model_bidir_google = Model(input=sequence, output=output)
# review model structure
print(model_bidir_google.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 30)            0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 30, 300)       3000000     input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNormal(None, 30, 300)       600         embedding_1[0][0]                
____________________________________________________________________________________________________
gru_1 (GRU)                      (None, 64)            70080       batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [17]:
# Bi-directional google
from keras.callbacks import ModelCheckpoint
model_bidir_google.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
start_time = time.time()


history_bidir_google = model_bidir_google.fit(X_train, y_train,
                    batch_size=batch_size,
                    nb_epoch=epochs,
                    validation_data=[X_test, y_test], 
                    verbose=2)

end_time = time.time()
average_time_per_epoch = (end_time - start_time) / epochs
print("avg sec per epoch:", average_time_per_epoch)


Train...
Train on 2780 samples, validate on 1390 samples
Epoch 1/16
46s - loss: 0.9940 - acc: 0.6129 - val_loss: 0.9379 - val_acc: 0.7201
Epoch 2/16
38s - loss: 0.9165 - acc: 0.6198 - val_loss: 0.8582 - val_acc: 0.7201
Epoch 3/16
48s - loss: 0.8347 - acc: 0.6266 - val_loss: 0.8077 - val_acc: 0.7201
Epoch 4/16
53s - loss: 0.7608 - acc: 0.6302 - val_loss: 0.7630 - val_acc: 0.7201
Epoch 5/16
52s - loss: 0.7035 - acc: 0.6601 - val_loss: 0.6987 - val_acc: 0.7187
Epoch 6/16
46s - loss: 0.6682 - acc: 0.6691 - val_loss: 0.6709 - val_acc: 0.7194
Epoch 7/16
60s - loss: 0.6360 - acc: 0.6863 - val_loss: 0.6642 - val_acc: 0.7187
Epoch 8/16
43s - loss: 0.6258 - acc: 0.6978 - val_loss: 0.6430 - val_acc: 0.7187
Epoch 9/16
15s - loss: 0.6136 - acc: 0.7119 - val_loss: 0.6155 - val_acc: 0.7187
Epoch 10/16
18s - loss: 0.6087 - acc: 0.7162 - val_loss: 0.6389 - val_acc: 0.7187
Epoch 11/16
49s - loss: 0.6093 - acc: 0.7183 - val_loss: 0.6067 - val_acc: 0.7187
Epoch 12/16
41s - loss: 0.6060 - acc: 0.7173 - val